In [4]:
# %pip install -q -U google-genai
# %pip install python-dotenv

import os
from dotenv import load_dotenv
from google import genai

load_dotenv()

True

In [5]:
# The client gets the API key from the environment variable `GEMINI_API_KEY`.
client = genai.Client()

response = client.models.generate_content(
    model="gemini-3-flash-preview", contents="Explain how AI works in a few words"
)
print(response.text)

AI finds patterns in data to make predictions.
